In [1]:
import os
os.remove('snorkel.db') #need these two files to exist in sandbox directory. Just copy from tutorials
os.system('cp snorkel.db\ corpus snorkel.db');

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession
session = SnorkelSession()

import sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

In [3]:
from snorkel.models import Corpus

corpus = session.query(Corpus).filter(Corpus.name == 'Hardware').one()
print "%s contains %d Documents" % (corpus, len(corpus))


Corpus (Hardware) contains 100 Documents


In [4]:
from snorkel.models import candidate_subclass
import re

Part = candidate_subclass('Part', ['part'])

from hardware_utils import OmniNgramsPart, OmniNgramsTemp

part_ngrams = OmniNgramsPart(n_max=3, split_tokens=None)

from hardware_utils import load_extended_parts_dict
from snorkel.matchers import DictionaryMatch, RegexMatchEach, RegexMatchSpan, Union

# filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
# parts_dict = load_extended_parts_dict(filename)
# print "Loaded %d part numbers." % len(parts_dict)
# parts_matcher = DictionaryMatch(d=parts_dict)

eeca_matcher = RegexMatchSpan(rgx='([b]{1}[abcdefklnpqruyz]{1}[\swxyz]?[0-9]{3,5}[\s]?[A-Z\/]{0,5}[0-9]?[A-Z]?([-][A-Z0-9]{1,7})?([-][A-Z0-9]{1,2})?)')
jedec_matcher = RegexMatchSpan(rgx='([123]N\d{3,4}[A-Z]{0,5}[0-9]?[A-Z]?)')
jis_matcher = RegexMatchSpan(rgx='(2S[abcdefghjkmqrstvz]{1}[\d]{2,4})')
others_matcher = RegexMatchSpan(rgx='((NSVBC|SMBT|MJ|MJE|MPS|MRF|RCA|TIP|ZTX|ZT|TIS|TIPL|DTC|MMBT|PZT){1}[\d]{2,4}[A-Z]{0,3}([-][A-Z0-9]{0,6})?([-][A-Z0-9]{0,1})?)')
parts_matcher = Union(eeca_matcher, jedec_matcher, jis_matcher, others_matcher)

In [5]:
from hardware_utils import get_gold_parts, get_gold_dict

gold_file = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
gold_parts = get_gold_parts(gold_file)

In [6]:
# Confirm that all gold parts are findable by regular expression
tp = []
fn = []
for g in gold_parts:
    if (re.match(eeca_matcher.rgx, g, flags=re.I) is not None 
        or re.match(jedec_matcher.rgx, g, flags=re.I)
        or re.match(jis_matcher.rgx, g, flags=re.I)
        or re.match(others_matcher.rgx, g, flags=re.I) is not None):
        tp.append(g)
    else:
        fn.append(g)
print len(tp), len(fn)

257 0


In [7]:
print tp[:5]
print '---------------------'
print fn[:5]

['BC817-16', 'BC857', 'BC856', 'BC849BW', '2N4124']
---------------------
[]


In [8]:
# import re
# from collections import defaultdict

# rgx = re.compile(r'\w*\d+(\-[\w]+|[A-Z]+[\w\-\/]*)')
# # rgx = re.compile(r'\w+-\w*-\w+')
# suffixes = defaultdict(int)
# print len(gold_parts)
# for g in gold_parts:
#     m = rgx.match(g)
#     if m:
#         suffixes[m.group(1)] += 1
#         if m.group(1)=='-25ZL1GA':
#             print g
# print len(suffixes)

In [9]:
# # from collections import Counter
# counter = Counter(suffixes.values())
# print counter
# print suffixes

In [10]:
from snorkel.candidates import CandidateExtractor

ce = CandidateExtractor(Part, [part_ngrams], [parts_matcher])

In [11]:
%time candidates = ce.extract(corpus.documents, 'Hardware Candidates', session)
print "%s contains %d Candidates" % (candidates, len(candidates))

[========================================] 100%

CPU times: user 42 s, sys: 456 ms, total: 42.5 s
Wall time: 42.4 s
Candidate Set (Hardware Candidates) contains 14174 Candidates


In [12]:
print len(candidates)
print candidates[0]

14174
Part(Span("MMBT3904", parent=3589, chars=[0,7], words=[0,0]))


In [13]:
for c in candidates[:3]: print c

Part(Span("MMBT3904", parent=3589, chars=[0,7], words=[0,0]))
Part(ImplicitSpan("MMBT3904", parent=44041, words=[0,0], position=[0]))
Part(Span("MMBT3904", parent=3661, chars=[0,7], words=[0,0]))


In [14]:
# TEMP
from hardware_utils import entity_level_total_recall

filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
(tp, fp, fn) = entity_level_total_recall(candidates, filename, 'stg_temp_min', relation=False)

Preparing candidates...
[========================================] 100%

Scoring on Entity-Level Total Recall
Entity-level Candidates extracted: 1358 
Entity-level Gold: 584
Intersection Candidates: 553
----------------------------------------
Overlap with Gold:  0.95

